## Lecture1:Colab（無料枠）で試せる、日本語LLM(youri7b,calm2-7b-chat)

Pythonを使えば、ChatGPTに代表されるLLM(大規模言語モデル)を動かして動作を確認することもできます（計算機の性能により動作しい場合もある）。


> 注）２つのモデルを切り替える際は「ランタイムを接続解除して削除」を行う必要がある場合があります。



### rinna/youri-7b

rinnaが開発した、meta社のLlama2_7Bをベースに日本語の学習データを用いて継続事前学習を行ったのが、汎用言語モデル「Youri 7B」です。

さらに、汎用言語モデルであるYouri 7Bに、対話形式でユーザーの指示を遂行するような追加学習をしたモデルが「Youri 7B Instruction」です。日本語の一問一答に応える能力が高くベンチマークにおいて高いスコアを達成します。

rinna/youri-7b-instruction-gptq　は、Youri 7B instructionを量子化してコンパクトにしたもので、これならば、日本語の性能がそそこでかつColab無料枠でも動作します。

### cyberagent/calm2-7b-chat
calm2-7b-chatは、CyberAgentが公開した日本語特化のLLM(大規模言語モデル)のopen-calmの対話モデルです。Colab無料枠で動かすために、量子化したmmnga/cyberagent-calm2-7b-chat-GPTQ-calib-ja-1kで動作を確認しています。



https://huggingface.co/rinna/youri-7b-instruction-gptq


In [ ]:
!pip install transformers
!pip install auto-gptq

In [ ]:
import torch
from transformers import AutoTokenizer
from auto_gptq import AutoGPTQForCausalLM

model_name = "rinna/youri-7b-instruction-gptq"
model_name = "mmnga/cyberagent-calm2-7b-chat-GPTQ-calib-ja-1k"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoGPTQForCausalLM.from_quantized(model_name, use_safetensors=True)

instruction = "次の日本語を英語に翻訳してください。"
input = "大規模言語モデル（だいきぼげんごモデル、英: large language model、LLM）は、多数のパラメータ（数千万から数十億）を持つ人工ニューラルネットワークで構成されるコンピュータ言語モデルで、膨大なラベルなしテキストを使用して自己教師あり学習または半教師あり学習によって訓練が行われる。"
prompt = f"""
以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示:
{instruction}

### 入力:
{input}

### 応答:
"""
token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        input_ids=token_ids.to(model.device),
        max_new_tokens=200,
        do_sample=True,
        temperature=0.5,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)


In [ ]:
def get_responce(instruction, input):

  prompt = f"""
  以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

  ### 指示:
  {instruction}

  ### 入力:
  {input}

  ### 応答:
  """
  token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

  with torch.no_grad():
      output_ids = model.generate(
          input_ids=token_ids.to(model.device),
          max_new_tokens=200,
          do_sample=True,
          temperature=0.5,
          pad_token_id=tokenizer.pad_token_id,
          bos_token_id=tokenizer.bos_token_id,
          eos_token_id=tokenizer.eos_token_id
      )

  output = tokenizer.decode(output_ids.tolist()[0])
  return output

In [ ]:
instruction = "あなたは優秀なAIです。的確に答えてください。"
input = "500円のケーキを３人で食べました、それぞれいくら払うべき？"

res = get_responce(instruction, input)
print(res)

In [ ]:
instruction = "あなたは優秀なAIです。的確に答えてください。"
input = "世界で一番高い山はなんですか？それは東京ドームより高いですか？"

res = get_responce(instruction, input)
print(res)

In [ ]:
instruction = "あなたは優秀なAIです。的確に答えてください。"
input = "清少納言というのは正式な名前か？"
res = get_responce(instruction, input)
print(res)